In [1]:
import pandas as pd
import numpy as np
from scipy.io import loadmat
from scipy.stats import ortho_group
from scipy.optimize import minimize

In [2]:
data = loadmat('ComGA-master/data/Flickr/Flickr.mat')

In [3]:
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Wed Sep 12 18:20:30 2018',
 '__version__': '1.0',
 '__globals__': [],
 'Network': <7575x7575 sparse matrix of type '<class 'numpy.float64'>'
 	with 482555 stored elements in Compressed Sparse Column format>,
 'Label': array([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]], dtype=uint8),
 'Attributes': <7575x12047 sparse matrix of type '<class 'numpy.float64'>'
 	with 225667 stored elements in Compressed Sparse Column format>,
 'Class': array([[8],
        [2],
        [7],
        ...,
        [3],
        [6],
        [4]], dtype=uint8)}

In [3]:
A = data['Network'].toarray()
n = A.shape[0]
X = data['Attributes'].toarray()
d =X.shape[1]
V = np.array([i for i in range(n)])

In [4]:
degree = np.zeros(n) # initialize list to hold values of degree

# calculate the sums along rows and sum along columns
colsum = A.sum(axis=0)
rowsum = A.sum(axis=1)

# loop through matrix and add up all degree connections
for j in range(n):
    degree[j] = colsum[j] + rowsum[j]

# get the diagonal entries to correct the for loop oversumming
B= A.diagonal()
b_flat = B.flat
diagMat = list(b_flat)


D = np.diag(degree - diagMat)

In [5]:
W = np.matmul(np.linalg.inv(D), A)

In [6]:
# Dissimilarity measure used is 0 if same class, 1 otherwise
W_x = np.zeros((n, n))
Classes = data['Class']
for i in range(n):
    for j in range(i, n):
        if Classes[i] != Classes[j]:
            W_x[i, j] = 1
            W_x[j, i] = 1

In [7]:
S = W + W_x

In [8]:
p = 8

In [9]:
M = np.matmul(np.linalg.matrix_power(W, p), X)

In [10]:
k = 2
lambda_ = 1

In [11]:
# Initializations
# B_nxk
B = np.delete(ortho_group.rvs(n), [i for i in range(k, n)], axis=1)
# Q_dxk
Q = np.delete(ortho_group.rvs(d), [i for i in range(k, d)], axis=1)
# Z_kxk
Z = ortho_group.rvs(k)

In [12]:
def calcul_G(S, B, n=n, k=k):
    G = np.zeros((n, k))
    B_swirl = np.matmul(S, B)
    for i in range(n):
        norm_diff = np.zeros(k)
        for k_dash in range(k):
            norm_diff[k_dash] = np.linalg.norm( B_swirl[i] - Z[k_dash])
        for k_iter in range(k):
            if k_iter == np.argmin(norm_diff):
                G[i][k_iter] = 1
    return G

In [13]:
def calcul_B(M, S, Q, G, Z):
    matrix_to_decomp = np.matmul(M, Q) + lambda_* np.matmul(S, np.matmul(G, Z))
    U_hat, Sigma_hat, V_hat_T = np.linalg.svd(matrix_to_decomp, full_matrices=False)
    return np.matmul(U_hat, V_hat_T)


In [14]:
def calcul_Q(M, B):
    return np.matmul(np.transpose(M), B)


In [15]:
def calcul_Z(G, S, B):
    matrix_to_decomp_Z = np.matmul(np.transpose(G), np.matmul(S, B))
    U, Sigma, V_T = np.linalg.svd(matrix_to_decomp_Z, full_matrices=False)
    return np.matmul(U, V_T)

In [16]:
def check_convergence(A, A_new, tol):
    return np.linalg.norm(A - A_new) < tol

In [19]:
def PAGEC(M, S, X, k = k, p = p, lambda_ = lambda_, tol=0.5):
    # Initializations
    # B_nxk
    B = np.delete(ortho_group.rvs(n), [i for i in range(k, n)], axis=1)
    # Q_dxk
    Q = np.delete(ortho_group.rvs(d), [i for i in range(k, d)], axis=1)
    # Z_kxk
    Z = ortho_group.rvs(k)

    converged = False

    G = calcul_G(S, B)
    while not converged:
        G_new = calcul_G(S, B)
        B_new = calcul_B(M, S, Q, G_new, Z)
        Q_new = calcul_Q(M, B)
        Z_new = calcul_Z(G_new, S, B_new)
        converged = all([check_convergence(G, G_new, tol),
                     check_convergence(B, B_new, tol),
                     check_convergence(Q, Q_new, tol),
                     check_convergence(Z, Z_new, tol)])


        G = G_new
        B = B_new
        Q = Q_new
        Z = Z_new

    return G, B, Q, Z


In [ ]:
G, B, Q, Z = PAGEC(M, S, X, k, p, lambda_)